[문제] Managing Conversation History
- trim_meassage()
- RunnablePassthrough
- itmegetter()

세션ID 설정하여, 대화 여러번 주고 받아야 함. 추가로 대화를 주고 받은 내역 출력.
LLM 모델이 과거 대화를 알지 못하는 상황을 만들기

In [1]:
## 1. 모듈 읽어오기
from langchain_core.runnables.history import RunnablePassthrough,RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory, BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_openai import ChatOpenAI
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

from dotenv import load_dotenv

## 2. 환경변수
load_dotenv()

True

In [2]:
## 3. 파일 읽기
file_name ='남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률(법률)(제20521호)(20250223).txt'

with open(file_name, 'r', encoding='utf-8') as file:
    law_file = file.read()
    print(law_file[:50])


남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률 ( 약칭: 남녀고용평등법 )
[시행 20


In [3]:
## 4. prompt 생성

template = [
    (
        'system',
          '''
        당신은 육아휴직 법률 전문가입니다.
        아래 문서를 참고하여, 사용자 질문에 성실하게 답변합니다.
        답변은 문서 내용을 기반으로 하되, 해당 조항도 표시합니다.
        - 문서: {law}
        '''),
        ('placeholder', '{chat_history}'),
        ('user', '{query}')
]

prompt = ChatPromptTemplate.from_messages(template)

In [4]:
## 5. 인스턴스 생성

llm = ChatOpenAI(
    model='gpt-4.1-nano',
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

llm

ChatOpenAI(callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x000001AE4A84BF10>], client=<openai.resources.chat.completions.completions.Completions object at 0x000001AE4A976350>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001AE4A9990F0>, root_client=<openai.OpenAI object at 0x000001AE47FA0370>, root_async_client=<openai.AsyncOpenAI object at 0x000001AE4A976410>, model_name='gpt-4.1-nano', model_kwargs={}, openai_api_key=SecretStr('**********'), streaming=True)

In [5]:
## 6. chain 구성
chain = prompt | llm 

In [6]:
## 7. 대화 이력 관리 인스턴스 생성
history = InMemoryChatMessageHistory()

store = {}

In [7]:
## 8.함수 정의: 대화 이력 인스턴스 리턴
def get_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [8]:
def trim_messages(messages, k=2):
    return messages[-k:]

In [9]:
## 10. chain에 대화 이력 기능 래핑해서 추가

with_message_history = RunnableWithMessageHistory(
    chain,
    get_history,
    history_messages_key='chat_history',
    input_messages_key='query',
)

In [ ]:
## 9. 실행 루프 (대화 반복 + 히스토리 제어)
law_file = "여기에 법령 내용 문자열 그대로 넣기 또는 파일에서 읽기"
session_id = "session_001"  # 고정 ID

while True:
    query = input("\n질문 입력 (또는 'exit' 입력 시 종료): ")

    if query.lower() == 'exit':
        break

    if query.lower() == 'history':
        history = get_history(session_id)
        print("\n📜 현재 히스토리:")
        for msg in history.messages:
            print(msg)
        continue

    if query.lower() == 'clear':
        get_history(session_id).clear()
        print("\n🧹 히스토리가 초기화되었습니다.")
        continue

    if query.lower().startswith('trim'):
        # 예: trim 1 → 최근 1개만 남김
        try:
            k = int(query.split()[1])
            trimmed = trim_messages(get_history(session_id).messages, k)
            get_history(session_id).messages = trimmed
            print(f"\n✂️ 최근 {k}개만 남기고 히스토리를 잘랐습니다.")
        except:
            print("⚠️ trim 명령은 'trim 숫자' 형식으로 입력하세요.")
        continue

    with_message_history.invoke(
        {"query": query, "law": law_file},
        config={"configurable": {"session_id": session_id}}
    )


남편이 사용할 수 있는 육아휴직 일수는 "휴직 개시일이 속하는 달의 다음달 말일까지 계속하여 1년 이내의 범위에서 1개월 이상" 신청할 수 있으며, 최대 1년(12개월)까지 사용할 수 있습니다. 이 경우, 육아휴직을 남편이 사용하는 것은 법률상 가능합니다.

구체적인 내용은 법령 제15조(육아휴직의 신청과 기간)에서 확인할 수 있으며, 근로자가 육아휴직을 신청하는 경우, 동일한 근로자가 최초 신청일로부터 1년 이내의 범위에서 연속 또는 불연속으로 육아휴직을 사용할 수 있다고 규정되어 있습니다. 

※ 참고: 육아휴직의 기간과 조건은 회사와의 협의에 따라 조정될 수 있으며, 별정 규정이나 개별 노동계약서 내용도 확인하시는 것이 좋습니다.

In [13]:
## 11. 실행
with_message_history.invoke(
    {'law': law_file, 'query':query},
    config={'configurable': {'session_id': 'abc'}}
)


남편이 사용할 수 있는 육아휴직일 수는 최대 12개월(365일)입니다. 이는 남성근로자가 육아휴직을 사용할 수 있는 기간에 대한 규정으로, 관련 법령에 따라 최대 12개월까지 육아휴직을 신청할 수 있습니다(관련 법령 조항 참고).

AIMessage(content='남편이 사용할 수 있는 육아휴직일 수는 최대 12개월(365일)입니다. 이는 남성근로자가 육아휴직을 사용할 수 있는 기간에 대한 규정으로, 관련 법령에 따라 최대 12개월까지 육아휴직을 신청할 수 있습니다(관련 법령 조항 참고).', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_38343a2f8f'}, id='run-bd33fcbe-843e-4f37-91bc-0e995e6bf4db-0')